In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import joblib

# Charger les données
df_vecteur = pd.read_csv('../../data/processed/clean_dataset.csv')

# Séparer les caractéristiques et la cible
X = df_vecteur['merged']
y = df_vecteur["prdtypecode"]

# Encoder la cible 
label_encoder = joblib.load('label_encoder.joblib')
y_encoded = label_encoder.transform(y)

# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Définir le pipeline pour l'entraînement
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_.astype(str)))

# Extraire les objets entraînés (sans SMOTE)
tfidf_trained = pipeline.named_steps['tfidf']
xgb_trained = pipeline.named_steps['xgb']

# Définir le pipeline pour l'inférence (ie sans SMOTE) et avec sklearn.pipeline pour la compatibilité avec le VotingClassifier
from sklearn.pipeline import Pipeline as SklearnPipeline

inference_pipeline = SklearnPipeline([
    ('tfidf', tfidf_trained),
    ('xgb', xgb_trained)
])

# Sauvegarde du pipeline complet
joblib.dump(inference_pipeline, 'inference_pipeline_xgb_model.joblib')

# Sauvegarde du label encoder
#joblib.dump(label_encoder, 'label_encoder.joblib')
